In [ ]:
# Paling Efektif
import cv2, numpy as np
from sklearn.cluster import KMeans

cap = cv2.VideoCapture(2)
cv2.namedWindow("ori-blur-bin", cv2.WINDOW_NORMAL)
cv2.resizeWindow("ori-blur-bin", (1280, 360))
cv2.createTrackbar("Kernel Size", "ori-blur-bin", 1, 31, lambda x: None)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    original_img = frame.copy()
    k_val = cv2.getTrackbarPos("Kernel Size", "ori-blur-bin")
    if k_val % 2 == 0:
        k_val += 1
    if k_val <= 0:
        k_val = 1
    blurred_img = cv2.GaussianBlur(frame, (k_val, k_val), 5)
    h, w = blurred_img.shape[:2]
    data = blurred_img.reshape(-1, 3).astype(np.float32)

    km = KMeans(n_clusters=2, random_state=42).fit(data)
    labels = km.labels_.reshape(h, w)
    centers = km.cluster_centers_

    bright = []
    for c in centers:
        b, g, r = c  # format BGR
        bright.append(0.114 * b + 0.587 * g + 0.299 * r)
    # Cluster dengan kecerahan terendah dianggap garis
    line_cluster = np.argmin(bright)

    # Pixel pada cluster garis = hitam (0), selainnya putih (255)
    mask = np.where(labels == line_cluster, 0, 255).astype(np.uint8)

    cv2.imshow("ori-blur-bin", np.hstack([original_img, blurred_img, cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)]))
    if cv2.waitKey(1) & 0xFF == ord("n"):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

cap = cv2.VideoCapture(1)
cv2.namedWindow("ori-blur-bin", cv2.WINDOW_NORMAL)
cv2.resizeWindow("ori-blur-bin", (1280, 360))
cv2.createTrackbar("Kernel Size", "ori-blur-bin", 1, 31, lambda x: None)
cv2.createTrackbar("Max Area", "ori-blur-bin", 100, 5000, lambda x: None)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    original_img = frame.copy()
    k_val = cv2.getTrackbarPos("Kernel Size", "ori-blur-bin")
    if k_val % 2 == 0:
        k_val += 1
    if k_val <= 0:
        k_val = 1

    blurred_img = cv2.GaussianBlur(frame, (k_val, k_val), 5)
    h, w = blurred_img.shape[:2]
    data = blurred_img.reshape(-1, 3).astype(np.float32)
    km = KMeans(n_clusters=2, random_state=42).fit(data)
    labels = km.labels_.reshape(h, w)
    centers = km.cluster_centers_

    bright = []
    for c in centers:
        b, g, r = c
        bright.append(0.114 * b + 0.587 * g + 0.299 * r)
    line_cluster = np.argmin(bright)

    # Jahitan=hitam (0), lainnya=putih (255)
    mask = np.where(labels == line_cluster, 0, 255).astype(np.uint8)

    # Ambil nilai Max Area dari slider
    max_area = cv2.getTrackbarPos("Max Area", "ori-blur-bin")

    # Tahap filtering komponen terlalu besar pada 'garis'
    # Karena connectedComponents biasa menganggap putih sebagai objek,
    # kita invert dulu mask (jahitan -> 255, latar -> 0)
    inv_mask = 255 - mask

    num_labels, lbls, stats, centroids = cv2.connectedComponentsWithStats(inv_mask, connectivity=8)
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area > max_area:
            # "Hilangkan" komponen besar ini dengan mengisinya 0 (background)
            inv_mask[lbls == i] = 0

    # Balikkan lagi sehingga jahitan tetap hitam
    filtered_mask = 255 - inv_mask

    cv2.imshow("ori-blur-bin", np.hstack([original_img, blurred_img, cv2.cvtColor(filtered_mask, cv2.COLOR_GRAY2BGR)]))

    if cv2.waitKey(1) & 0xFF == ord("n"):
        break

cap.release()
cv2.destroyAllWindows()

# Connection Break Detection

In [ ]:
import cv2, numpy as np
from sklearn.cluster import KMeans
import cvzone


def group_bboxes_by_row(bboxes, overlap_thresh=10):
    rows = []
    for bb in bboxes:
        l, t, r, b = bb
        placed = False
        for row in rows:
            # Cek overlap vertikal antara bb dan row pertama
            # ambil bounding box referensi di row (misal yang pertama di row itu)
            l0, t0, r0, b0 = row[0]
            # Asumsi overlap jika selisih vertikal tidak terlalu besar
            # atau setidaknya ada irisan.
            # Cara sederhana: ambil height min, dsb.
            # Atau ambil bounding box paling atas di row. Terserah definisi.
            # Di sini kita cek jika (b < t0 - overlap_thresh) atau (t > b0 + overlap_thresh) => tidak overlap
            # Sebaliknya jika ada overlap => tambahkan bb ke row
            if not (b < t0 - overlap_thresh or t > b0 + overlap_thresh):
                row.append(bb)
                placed = True
                break
        if not placed:
            rows.append([bb])
    return rows


def detect_c_breaks_bbox_horizontal(frame, mask, gap_threshold=20):
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    num_labels, labels_cc, stats, _ = cv2.connectedComponentsWithStats(mask, 8)
    bboxes = []
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > 10:
            bboxes.append((x, y, x + w, y + h))  # (left, top, right, bottom)

    if len(bboxes) <= 1:
        return frame, False

    # 1. Kelompokkan bboxes ke "baris" sesuai overlap vertikal
    rows = group_bboxes_by_row(bboxes, overlap_thresh=10)

    putus = False
    # 2. Deteksi gap horizontal di setiap baris
    for row in rows:
        # sort di baris ini menurut x
        row.sort(key=lambda b: b[0])
        # cek jarak antar bounding box di baris ini
        for i in range(len(row) - 1):
            lA, tA, rA, bA = row[i]
            lB, tB, rB, bB = row[i + 1]

            # gap horizontal
            distance = lB - rA
            if distance > gap_threshold:
                putus = True
                mx_gap = rA + distance // 2
                # Tentukan y-gap misal di tengah dua bboxes
                my_gap = (tA + bA) // 2
                cv2.circle(frame, (mx_gap, my_gap), 10, (0, 0, 255), 3)

    return frame, putus


# -- KODE UTAMA ---
cap = cv2.VideoCapture(2)
cv2.namedWindow("ori-blur-bin", cv2.WINDOW_NORMAL)
cv2.resizeWindow("ori-blur-bin", (1280, 360))
cv2.createTrackbar("Kernel Size", "ori-blur-bin", 1, 31, lambda x: None)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    original_img = frame.copy()
    k_val = cv2.getTrackbarPos("Kernel Size", "ori-blur-bin")
    if k_val % 2 == 0:
        k_val += 1
    if k_val <= 0:
        k_val = 1
    blurred_img = cv2.GaussianBlur(frame, (k_val, k_val), 5)

    h, w = blurred_img.shape[:2]
    data = blurred_img.reshape(-1, 3).astype(np.float32)
    km = KMeans(n_clusters=2, random_state=42).fit(data)
    labels = km.labels_.reshape(h, w)
    centers = km.cluster_centers_

    # Cari cluster paling gelap => diasumsikan 'jahitan'
    brightness = []
    for c in centers:
        b, g, r = c
        gray_approx = 0.114 * b + 0.587 * g + 0.299 * r
        brightness.append(gray_approx)
    darkest_cluster = np.argmin(brightness)

    # Buat mask jahitan=0, background=255
    mask = np.where(labels == darkest_cluster, 0, 255).astype(np.uint8)

    result_frame, is_broken = detect_c_breaks_bbox_horizontal(original_img, mask, gap_threshold=30)
    if is_broken:
        cvzone.putTextRect(result_frame, "Reject: Ada Jahitan Terlewat", (20, 40), 1, 2, offset=5, border=2, colorR=(0, 255, 255), colorT=(0, 0, 0), colorB=(255, 255, 255))
    else:
        cvzone.putTextRect(result_frame, "Good: Tidak Ada Jahitan Terlewat", (20, 40), 1, 2, offset=5, border=2, colorR=(0, 255, 0), colorT=(0, 0, 0), colorB=(255, 255, 255))

    cv2.imshow("ori-blur-bin", np.hstack([result_frame, blurred_img, cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)]))
    if cv2.waitKey(1) & 0xFF == ord("n"):
        break

cap.release()
cv2.destroyAllWindows()